In [1]:
import cv2
import pafy
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.models import model_from_json 
from matplotlib import pyplot as plt

In [2]:
#Function to detect the expression and show it along the data passed
def pre():
        while(True):
            ret,img=cap.read()    
            
            #Converting the input stream to grayscale
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            
            #Detecting the faces from the grayscale stream
            faces= face_cascade.detectMultiScale(gray,1.3,5)
            
            #Calssifying the expression on the face and tagging the expression near the face
            for(x,y,w,h) in faces:
                
                #crop the face
                face = img[int(y):int(y+h), int(x):int(x+w)] 
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) 
                
                #resize the face
                face = cv2.resize(face, (48, 48))
                
                #Reshaping and Scaling
                img_pixels=image.img_to_array(face)
                img_pixels=np.expand_dims(img_pixels, axis=0)
                img_pixels /=255
                
                #Detecting the expression in the face
                predictions=model.predict(img_pixels)        
                max_index = np.argmax(predictions[0])        
                emotion=emotions[max_index]
                
                #Tagging the expression
                cv2.putText(img,emotion,(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
            
            #Showing the real time feed with the expression tagged along the face
            cv2.imshow('img',img)
            if (cv2.waitKey(10))==27:
                break
        cap.release()
        cv2.destroyAllWindows()

In [3]:
#Loading the model
model= model_from_json(open("./modeladam.json","r").read())
model.load_weights('./modeladam.h5')

emotions = ('angry','disgust','fear','happy','sad','surprise','neutral')

face_cascade= cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

#Getting the input from the user
inp=int(input("Enter 1 for Live Cam Facial Emotion Detection, 2 for Facial Emotion Detection in a YouTube Video and 3 for a image\r\n"))
#Function for FED to work on front cam
if inp==1:
        cap=cv2.VideoCapture(0)
        pre()
#Function for FED to work on youtube videos
elif inp==2:
        url=input("Enter the link of the YouTube video\r\n")
        video=pafy.new(url)
        best=video.getbest(preftype="mp4")
        cap=cv2.VideoCapture(best.url)
        pre()
        cap.release()
        cv2.destroyAllWindows()
#Function to for FED to work on pics
elif inp==3:
        filepath=input("Enter the local file path of the image\r\n")
        img=image.load_img(filepath,grayscale=True,target_size=(48,48))
        x=image.img_to_array(img)
        x=np.expand_dims(x,axis=0)
        x=x/255
        detect= model.predict(x)
        
        y_pos = np.arange(len(emotions))
    
        plt.bar(y_pos, detect[0], align='center', alpha=0.5)
        plt.xticks(y_pos, emotions)
        plt.ylabel('percentage')
        plt.title('emotion')
        
        plt.show()
        
        x = np.array(x, 'float32')
        x = x.reshape([48, 48]);
        x=x*255
        plt.gray()
        plt.imshow(x)
        plt.show()
        
else:
        print("Invalid input")

Enter 1 for Live Cam Facial Emotion Detection, 2 for Facial Emotion Detection in a YouTube Video and 3 for a image
1
